<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/Language_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

--2023-06-12 09:45:19--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.128, 142.251.172.128, 142.250.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip.1’

spa-eng.zip.1       100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2023-06-12 09:45:19 (242 MB/s) - ‘spa-eng.zip.1’ saved [2638744/2638744]



In [2]:
!unzip -q spa-eng.zip

replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace spa-eng/spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [3]:
text_file = "spa-eng/spa.txt"

In [4]:
with open(text_file) as f:
  lines = f.read().split("\n")[:-1] 

In [5]:
text_pairs = []

In [6]:
#with open("spa-eng/san.txt", 'w') as w:
#  w.writelines("Santosh sarangi \n123 780 \nHello \nZebra animal")

In [7]:
#with open("spa-eng/san.txt") as f:
#  lines = f.read().split("\n")

In [8]:
len(lines)

118964

In [9]:
for line in lines:
  english, spanish = line.split("\t")
  spanish = "[start]" + spanish + "[end]"
  text_pairs.append((english, spanish))

In [39]:
import random
print(random.choice(text_pairs))

("Didn't you hear the doorbell?", '[start]¿No escuchaste el timbre?[end]')


In [40]:
random.shuffle(text_pairs)

In [41]:
num_val_samples = int(0.15 * len(text_pairs))

In [42]:
num_train_samples = len(text_pairs) - 2 * num_val_samples

In [43]:
train_pairs = text_pairs[:num_train_samples]

In [44]:
val_pairs = text_pairs[num_train_samples: num_train_samples+num_val_samples]

In [45]:
test_pairs = text_pairs[num_train_samples+num_val_samples:]

In [17]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [18]:
import string

In [19]:
import re

In [20]:
strip_chars = string.punctuation + "¿"

In [21]:
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


In [22]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)

  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(strip_chars)}]", ""
  )

In [23]:
vocab_size = 25000
sequence_length = 20

In [46]:
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode = "int",
    output_sequence_length=sequence_length
)

In [47]:
target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization
)

In [48]:
train_english_text = [pair[0] for pair in train_pairs]

In [49]:
train_spanish_text = [pair[1] for pair in train_pairs]

In [50]:
source_vectorization.adapt(train_english_text)

In [51]:
target_vectorization.adapt(train_spanish_text)

In [52]:
batch_size = 64

In [53]:
def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({"english": eng, "spanish": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [54]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["english"].shape: {inputs["english"].shape}')
    print(f'inputs["spanish"].shape: {inputs["spanish"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["english"].shape: (64, 20)
inputs["spanish"].shape: (64, 20)
targets.shape: (64, 20)


In [55]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [61]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [62]:
epochs = 1  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_8 (Positi  (None, None, 256)   6405120     ['english[0][0]']                
 onalEmbedding)                                                                                   
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_4 (Transfo  (None, None, 256)   3155456     ['positional_embedding_